In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from datetime import datetime
import warnings

plt.rc('font', family='malgun gothic')
plt.rc('axes' , unicode_minus = False)
warnings.filterwarnings(action = 'ignore')

In [14]:
df = pd.read_parquet("C:/Users/whileduck/Desktop/Github/Concert-Hall-Price-Model/data/거래까지걸린시간_추가_클래식_데이터.parquet")
seat_information = pd.read_csv("C:/Users/whileduck/Desktop/Github/Concert-Hall-Price-Model/data/seat_information_angle_ver3.csv")
display(df.head())
print(df.shape)

,seat,X,Y,Z,층,블록,열,넘버,무대까지의 거리,좌우면적시야각,...,원등급추정,누적검색량,콘서트홀클래식_누적검색량,예술의전당클래식_누적검색량,서울클래식_누적검색량,좌석 오픈 여부,거래까지걸린시간,거래까지걸린시간_시간,거래까지걸린시간_등수,표준화 등수 점수
0,1층 A블록1열 1,1451,542,-93,1층,A블록,1,1,1551.713247,15.219900,...,2,1050.0,1000.0,0.0,50.0,1,19 days 15:06:00,471.1,38.0,0.339286
1,1층 A블록1열 2,1406,555,-93,1층,A블록,1,2,1514.433888,16.945594,...,2,1050.0,1000.0,0.0,50.0,1,19 days 15:06:00,471.1,38.0,0.339286
2,1층 A블록1열 3,1361,568,-93,1층,A블록,1,3,1477.698887,18.869853,...,2,1050.0,1000.0,0.0,50.0,1,19 days 15:06:00,471.1,38.0,0.339286
3,1층 A블록1열 4,1315,580,-93,1층,A블록,1,4,1440.234009,21.009736,...,2,1050.0,1000.0,0.0,50.0,1,19 days 15:06:00,471.1,38.0,0.339286
4,1층 A블록1열 5,1270,591,-93,1층,A블록,1,5,1403.862529,23.314833,...,2,1050.0,1000.0,0.0,50.0,1,19 days 15:06:00,471.1,38.0,0.339286


(378255, 54)


# **데이터 테이블 생성 (공연 별, 좌석 별)**

In [15]:
reservation_grouping = df.groupby('전체공연시간').agg({'좌석 오픈 여부' : np.sum, 
                                    '예매여부' : np.sum, '원가격추정':[np.min , np.mean , np.max],'거래까지걸린시간_시간' : [np.min,np.mean,np.max]}).reset_index()
reservation_grouping['공연 예매율'] = reservation_grouping['예매여부'] / reservation_grouping['좌석 오픈 여부'] # 공연 별 예매율 계산 

reservation_grouping = reservation_grouping.rename(columns = {'price': '공연 별 평균 가격'})

reservation_grouping.columns = [' '.join(col).strip() for col in reservation_grouping.columns.values]

display(reservation_grouping.sample(5))
print(reservation_grouping.shape)

,전체공연시간,좌석 오픈 여부 sum,예매여부 sum,원가격추정 amin,원가격추정 mean,원가격추정 amax,거래까지걸린시간_시간 amin,거래까지걸린시간_시간 mean,거래까지걸린시간_시간 amax,공연 예매율
144,2023-04-16 17:00:00,2505,245,25000,74075.848303,95000,14.000000,359.459728,1240.966667,0.097804
52,2020-06-03 19:30:00,2142,622,20000,45447.105788,80000,326.033333,4275.988585,4557.083333,0.290383
110,2022-09-15 20:00:00,2505,2087,30000,44075.848303,70000,9.000000,874.780011,907.850000,0.833134
95,2022-06-09 19:30:00,2325,268,30000,53399.201597,80000,16.016667,650.624192,1052.016667,0.115269
98,2022-06-30 19:30:00,2505,1396,30000,59065.868263,90000,14.016667,272.822194,379.400000,0.557285


(151, 10)


In [16]:
# 공연별 예매율 테이블 생성

cols = ['play_date', 'play_st_time','pre_open_date','open_date', 'running_time', 
        'intermission','전체공연시간','콘서트홀클래식_누적검색량','예술의전당클래식_누적검색량', '서울클래식_누적검색량'] # 합병해주자 


pre_table = pd.merge(reservation_grouping,df[cols], on = '전체공연시간', how = 'left').drop_duplicates() # merge 후 중복을 제거 

reservation_table = pre_table.reset_index(drop = True) # index 초기화 

In [17]:
# 공연별 요약 테이블에서 EDA 할 때 용이하도록 날짜도 포함해서 만들도록 하자 

reservation_table['연도'] = pd.to_datetime(reservation_grouping['전체공연시간']).map(lambda x: x.year)
reservation_table['월'] = pd.to_datetime(reservation_grouping['전체공연시간']).map(lambda x: x.month)
reservation_table['연 월'] = pd.to_datetime(reservation_table['연도'].astype(str) + '-' + reservation_table['월'].astype(str)) 
reservation_table['일'] = pd.to_datetime(reservation_grouping['전체공연시간']).map(lambda x: x.day)
reservation_table['시간'] = pd.to_datetime(reservation_grouping['전체공연시간']).map(lambda x: x.hour)
reservation_table['요일'] =  pd.to_datetime(reservation_table['play_date']).map(lambda x: x.weekday())  # 월요일 0 , 화요일 1 .... 일요일 6 으로 맵핑되도록 변경

In [18]:
reservation_table['총_누적_검색량'] = reservation_table['콘서트홀클래식_누적검색량'] + reservation_table['예술의전당클래식_누적검색량'] + reservation_table['서울클래식_누적검색량']

In [19]:
display(reservation_table.sample(5))
print(reservation_table.shape)

,전체공연시간,좌석 오픈 여부 sum,예매여부 sum,원가격추정 amin,원가격추정 mean,원가격추정 amax,거래까지걸린시간_시간 amin,거래까지걸린시간_시간 mean,거래까지걸린시간_시간 amax,공연 예매율,...,콘서트홀클래식_누적검색량,예술의전당클래식_누적검색량,서울클래식_누적검색량,연도,월,연 월,일,시간,요일,총_누적_검색량
56,2020-10-20 19:30:00,1686,803,15000,33836.327345,60000,14.033333,499.197323,643.900000,0.476275,...,270.0,0.0,10.0,2020,10,2020-10-01,20,19,1,280.0
61,2021-03-20 17:00:00,1891,469,5000,50363.273453,70000,16.000000,426.184684,929.816667,0.248017,...,290.0,0.0,0.0,2021,3,2021-03-01,20,17,5,290.0
118,2022-10-29 17:00:00,2505,1786,5000,59626.746507,110000,14.000000,1676.331859,2105.166667,0.712974,...,640.0,0.0,150.0,2022,10,2022-10-01,29,17,5,790.0
58,2020-12-19 17:00:00,1384,1106,20000,35592.814371,50000,17.466667,322.325362,979.566667,0.799133,...,1420.0,0.0,20.0,2020,12,2020-12-01,19,17,5,1440.0
148,2023-05-23 19:30:00,2505,1698,30000,80457.085828,110000,15.000000,935.243375,2012.700000,0.677844,...,850.0,40.0,40.0,2023,5,2023-05-01,23,19,1,930.0


(151, 26)


# **좌석 데이터 요약 테이블 생성**

In [20]:
seat_grouping = df.groupby('seat').agg({'예매여부': np.sum, '좌석 오픈 여부': np.sum, 
                                        '원가격추정':np.mean, '거래까지걸린시간_시간':np.mean,
                                        '거래까지걸린시간_등수' : np.mean, '거래까지걸린시간_등수':[np.max, np.min],
                                        '표준화 등수 점수': np.mean}).reset_index()

seat_grouping['좌석 예매율'] = seat_grouping['예매여부'] / seat_grouping['좌석 오픈 여부']
seat_grouping = seat_grouping.rename(columns = {'price': '좌석 별 평균 가격'})


seat_grouping.columns = [' '.join(col).strip() for col in seat_grouping.columns.values]
pre_table = seat_grouping.reset_index(drop = True)

pre_table.sample(5)

,seat,예매여부 sum,좌석 오픈 여부 sum,원가격추정 mean,거래까지걸린시간_시간 mean,거래까지걸린시간_등수 amax,거래까지걸린시간_등수 amin,표준화 등수 점수 mean,좌석 예매율
1486,2층 B블록7열 12,81,142,71920.529801,732.837449,967.0,3.0,0.294512,0.570423
1308,2층 A블록5열 11,77,144,64006.622517,1012.451515,749.0,4.0,0.212813,0.534722
993,1층 D블록7열 11,92,149,99900.662252,850.821920,985.0,3.0,0.268282,0.617450
1805,3층 A블록1열 4,64,145,43642.384106,871.014583,967.0,3.0,0.213984,0.441379
1134,1층 E블록20열 3,68,131,73576.158940,818.003186,985.0,2.0,0.247874,0.519084


In [21]:
# merge 하기 전에 이름을 변경해주자

seat_information = seat_information.rename(columns = {'전체_좌석' : 'seat'})

seat_table = pd.merge(pre_table, seat_information, how = 'left', on = 'seat')

display(seat_table.sample(5))

,seat,예매여부 sum,좌석 오픈 여부 sum,원가격추정 mean,거래까지걸린시간_시간 mean,거래까지걸린시간_등수 amax,거래까지걸린시간_등수 amin,표준화 등수 점수 mean,좌석 예매율,Unnamed: 0,...,무대까지의 거리,좌우면적시야각,상하면적시야각,시야방해점수,좌석공간넓음,휠체어석,1/거리**3,무대정면과의각도,좌우시선각도,상하시선각도
737,1층 C블록7열 9,92,139,111357.615894,760.144384,908.0,1.0,0.364894,0.661871,336,...,1321.514283,67.536139,3.628259,100,0,0,4.332957e-10,0.996,5.210131,0.520281
1850,3층 A블록6열 7,62,140,42052.980132,689.656183,746.0,2.0,0.239826,0.442857,2336,...,3217.969857,23.364184,11.783913,94,0,0,3.000918e-11,0.839,13.479288,25.551858
2454,합창석 H블록2열 14,38,150,41092.715232,857.802632,890.0,3.0,0.111543,0.253333,1358,...,1210.696081,40.692782,51.226837,89,0,0,5.635009e-10,-0.310,0.039321,16.803477
1707,2층 E블록3열 14,66,147,62350.993377,859.125253,985.0,7.0,0.193897,0.448980,1702,...,2556.732485,26.480381,15.220813,96,0,0,5.983346e-11,0.760,6.040770,16.286232
429,1층 B블록6열 2,91,139,105198.675497,923.208425,908.0,1.0,0.319920,0.654676,263,...,1495.972593,46.789581,23.207038,100,0,0,2.986958e-10,0.761,32.746990,0.268101


# **추후 특정 기준에 따라 요약 테이블을 생성하기 위해 함수로 만들어두자**

In [22]:
class ConditionSummaryTable:
    
    def __init__(self,seat_information):
        
        self.seat_information = seat_information
        self.seat_information = self.seat_information.rename(columns = {'전체_좌석' : 'seat'})
        
    def reservation_condition_table(self ,cond_df):
        
        
        self.reservation_grouping = cond_df.groupby('전체공연시간').agg({'좌석 오픈 여부' : np.sum, 
                                    '예매여부' : np.sum, '원가격추정':[np.min, np.mean, np.max],'거래까지걸린시간_시간' : [np.min,np.mean,np.max]}).reset_index()
        
        
        self.reservation_grouping['공연 예매율'] = self.reservation_grouping['예매여부'] / self.reservation_grouping['좌석 오픈 여부'] # 공연 별 예매율 계산 

        self.reservation_grouping = self.reservation_grouping.rename(columns = {'price': '공연 별 평균 가격'})

        self.reservation_grouping.columns = [' '.join(col).strip() for col in self.reservation_grouping.columns.values]

        # 공연별 예매율 테이블 생성

        cols = ['play_date', 'play_st_time','pre_open_date','open_date', 'running_time', 
                'intermission','전체공연시간','콘서트홀클래식_누적검색량','예술의전당클래식_누적검색량', '서울클래식_누적검색량'] # 합병해주자 


        pre_table = pd.merge(reservation_grouping,cond_df[cols], on = '전체공연시간', how = 'left').drop_duplicates() # merge 후 중복을 제거 

        self.reservation_table = pre_table.reset_index(drop = True) # index 초기화 
        
        # 공연별 요약 테이블에서 EDA 할 때 용이하도록 날짜도 포함해서 만들도록 하자 

        self.reservation_table['연도'] = pd.to_datetime(self.reservation_table['전체공연시간']).map(lambda x: x.year)
        self.reservation_table['월'] = pd.to_datetime(self.reservation_table['전체공연시간']).map(lambda x: x.month)
        self.reservation_table['연 월'] = pd.to_datetime(self.reservation_table['연도'].astype(str) + '-' + reservation_table['월'].astype(str)) 
        self.reservation_table['일'] = pd.to_datetime(self.reservation_table['전체공연시간']).map(lambda x: x.day)
        self.reservation_table['시간'] = pd.to_datetime(self.reservation_table['전체공연시간']).map(lambda x: x.hour)
        self.reservation_table['요일'] =  pd.to_datetime(self.reservation_table['전체공연시간']).map(lambda x: x.weekday())  # 월요일 0 , 화요일 1 .... 일요일 6 으로 맵핑되도록 변경
        
        return self.reservation_table
    
    def seat_information_condition_table(self,cond_df):
        
        
        self.seat_grouping = cond_df.groupby('seat').agg({'예매여부': np.sum, '좌석 오픈 여부': np.sum, 
                                        '원가격추정':np.mean, '거래까지걸린시간_시간':np.mean,
                                        '거래까지걸린시간_등수' : np.mean, '거래까지걸린시간_등수':[np.max, np.min],
                                        '표준화 등수 점수': np.mean}).reset_index()


        self.seat_grouping['좌석 예매율'] = self.seat_grouping['예매여부'] / self.seat_grouping['좌석 오픈 여부']
        self.seat_grouping = self.seat_grouping.rename(columns = {'price': '좌석 별 평균 가격'})


        self.seat_grouping.columns = [' '.join(col).strip() for col in self.seat_grouping.columns.values]
        
        self.pre_table = self.seat_grouping.reset_index(drop = True)
        

        self.seat_table = pd.merge(pre_table, self.seat_information, how = 'left', on = 'seat')
        
        return self.seat_table

In [24]:
file_path = 'C:/Users/whileduck/Desktop/Github/Concert-Hall-Price-Model/data/'

reservation_table.to_parquet(file_path + 'reservation_table.parquet',index = False)
seat_table.to_parquet(file_path + 'seat_table.parquet', index = False)